#                              Algoritmi za računanje SVD dekompozicije


U rešavanju problema linearne algebre, dekompozicija matrice predstavlja važnu ulogu. Jedna od poznatijih dekompozicija je <b>singularna dekompozicija</b> (skraćeno, SVD), čijim izračunavanjem smo se bavili u našem radu i prikazali dva algoritma za efikasno izračunavanje ove dekompozicije. 

Za datu matricu A, dimenzije $mxn$, singularna dekompozicija matrice A je $$ A =  UΣV^T $$
Matrica U je ortogonalna i za nju važi da je dimenzije $mxm$. Matrica V je takođe ortogonalna i dimenzija je $nxn$. Matrica Σ je dimenzija $mxn$ i ima nenegativne elemente na glavnoj dijagonali, dok su ostali elementi te matrice nule.
Dijagonalni elementi matrice Σ se nazivaju <b>singularnim vrednostima matrice A</b> i broj singularnih vrednosti koje su nenula predstavlja rang matrice A. Važi da su singularne vrednosti jednake kvadratnim korenima sopstvenih vrednosti matrica $A^TA$ i $AA^T$. Kolone matrice U predstavljaju <i>leve singularne vektore</i> matrice A, odnosno sopstvene vektore matrice $AA^T$, dok kolone matrice V predstavljaju <i>desne singularne vektore</i>, odnosno sopstvene vektore matrice $A^TA$. Matrica Σ ima $m-n$ vrsta koje se sastoje samo iz nula, pa je moguće i drugačije razložiti matricu A : $$ A =  U'Σ'V^T $$
pri čemu važi da je U' dimenzija $mxn$, Σ' dimenzija $nxn$.

Prednosti SVD dekompozicije:
- SVD dekompozicija je pogodna za korišćenje pri rešavanju sistema jednačina u slučaju matrica koje su bliske singularnim. 
- Izračunavanje singularne dekompozicije je numerički vrlo stabilno.
- Pomoću SVD dekompozicije moguće je izvesti ortogonalizaciju skupa vektora.
- Inverz matrice A se lako računa ukoliko je poznata njena SVD dekompozicija, s obzirom da su matrice U i V ortogonalne.




Uopšteno, algoritmi za izračunavanje singularnih vrednosti su analogni algoritmima za izračunavanje sopstvenih vrednosti
simetričnih matrica. 
Oba algoritma pretpostavljaju da je  $m ≥ n$, a ukoliko je $m < n$, može se računati SVD dekompozicija matrice $A^T$. Da bi se izbegla nepotrebna komplikacija, takođe algoritmi će biti predstavljeni na realnim matricama. 

# Implementacija prvog algoritma

Prvi algoritam koji smo impelementirali se naziva Biorthogonalizacija SVD, a takođe je poznat pod nazivom <b>jednosmerna Jakobijeva metoda za SVD </b> (eng. One-sided Jacobi method for SVD). 
Kao ulaz prima $m$, $n$, A gde je A $m x n$.
Izlaz predstavljaju matrice U, Σ, V gde je Σ dijagonalna matrica, U and V imaju ortonormirane kolone i U je reda $m × n$, a V je reda $n × n$. Važi da je $$ A = UΣV^T $$.

Algoritam implicitno racuna vrednost $A^TA$, a zatim koristi Jakobijeve rotacije da napravi dijagonalnu matricu. Jakobijeva rotacija jeste 2x2 matrica koja dijagonalizuje simetričnu $2x2$ matricu. Treba pronaći ugao rotacije θ i matricu rotacije $Θ = \begin{bmatrix}cosθ  & -sinθ \\sinθ  & cosθ \end{bmatrix}$ za datu matricu $A^TA = \begin{bmatrix}α & γ\\γ & β\end{bmatrix}$ tako da važi $Θ^TA^TAΘ = D$ gde je D dijagonalna matrica. Ako definišemo $τ = (β − α) / 2γ $ i $t = sinθ/cosθ$, nailazimo na kvadratnu jednačinu $t^2 + 2τt − 1 = 0$. Iz ove kvadratne jednačine uzimamo rešenje $t$ koje je manje po apsolutnoj vrednosti i koristeći jednakosti $t = sinθ/cosθ$ i $sinθ^2+cosθ^2=1$, dolazimo do potrebnih vrednosti $cosθ =1/sqrt(1 + t^2)$ i $sinθ = c*t$. Ovo je dovoljno, nema potrebe da se računa vrednost samog θ.

Algoritam prolazi kroz implicitno konstruisanu matricu $A^TA$, bira i i j, zatim računa 2x2 matricu koju Jakobijevom rotacijom dijagonalizuje. Postupak se ponavlja do konvergencije. U matrici V se nalazi akumulacija svih rotacija kolona. Kada dođemo do kraja algoritma, matrica Σ jeste implicitno generisana. Singularne vrednosti predstavljaju norme kolona rezultujuće matrice U, a leve singularne vektore računamo kao normalizovane kolone rezultujućeg U. 

Algoritam ne vraća sortirane singularne vrednosti. Kada bi se one sortirale, trebalo bi permutovati odgovarajuće kolone matrica U i V.

In [9]:
import copy
import numpy as np
from scipy import linalg as la

def SVD(A, m, n, eps = 1.e-4):
    # 1. korak, U = A
    U = np.array(copy.copy(A))
    # 2. korak, V = I(nxn)
    V = np.eye(n) 
    # 3. korak, izračunamo N^2 = M, s = 0, first = true
    M = 0
    s = 0
    first = True
    for i in range(m):
        for j in range(n):
            M += U[i][j] * U[i][j]

    # korak 4
    a = 0
    b = 0
    c = 0
    error = 1
    while np.sqrt(s) > eps*eps*M or first == True:
        # 4.a
        s = 0
        first = False
        
        # 4.b
        for i in range(n-1):
            for j in range(i+1, n):
                squaresum = 0
                a = 0
                b = 0
                c = 0
                # izračunamo c,s,d1,d2
                for k in range(m):
                    squaresum += U[k][i]*U[k][j]
                    a += U[k][i]*U[k][i]
                    b += U[k][j]*U[k][j]
                    c += U[k][i]*U[k][j]
                s += squaresum * squaresum

                # Jacobijeva rotacija
                tau = ((b - a)* 1.0) / (2*c)
                t = np.sign(tau) * 1.0/(np.abs(tau) + np.sqrt(1 + tau*tau))
                cs = 1.0/np.sqrt(1 + t*t)
                sn = cs*t
                
                # apdejtujemo kolone i and j matrice U
                for k in range(m):
                    tmp = U[k][i]
                    U[k][i] = cs*tmp - sn*U[k][j]
                    U[k][j] = sn*tmp + cs*U[k][j]
                    
                # apdejtujemo matricu V
                for k in range(n):
                    tmp = V[k][i]
                    V[k][i] = cs*tmp - sn*V[k][j]
                    V[k][j] = sn*tmp + cs*V[k][j]
                
    # korak 5
    b = np.zeros(n)
    for i in range(n):
        b[i] = np.linalg.norm(U[:,i])
        for k in range(m):
            U[k][i] = U[k][i] / b[i]

    E = np.diag(b)
    
    
    print("Matrica U:")
    print(U)
    print("Matrica E:")
    print(E)
    print("Matrica V.T:")
    print(V.T)
    
    print("Matrica A:")
    print(np.dot(U, np.dot(E, V.T)))

# Primeri

In [8]:
A = [ [1.0 ,3.0,2.0],
       [5.0 ,6.0,4.0],
       [7.0 ,8.0,9.0],
        [2.0,1.0,6.0]]
U, S, V = la.svd(A, full_matrices = False)
print("Izlaz ugradjene funkcije je:")
print()
print("Matrica U:")
print(U)
print("Matrica S:")
print(S)
print("Matrica V:")
print(V)
print()
print("Izlaz implementirane funkcije je:")
print()
SVD(A, 4, 3)

Izlaz ugradjene funkcije je:

Matrica U:
[[-2.00416656e-01  1.64922791e-01  9.46812646e-01]
 [-4.86189124e-01  4.93693633e-01 -3.18829561e-01]
 [-7.92156024e-01 -1.08891970e-02 -4.35050639e-02]
 [-3.09745271e-01 -8.53784806e-01 -9.12882764e-04]]
Matrica S:
[17.58269051  3.94408679  1.13717809]
Matrica V:
[[-0.50026161 -0.57814691 -0.64458085]
 [ 0.21541031  0.63792506 -0.73935784]
 [-0.83865173  0.5087217   0.19459061]]

Izlaz implementirane funkcije je:

Matrica U:
[[-9.46812646e-01  2.00416656e-01 -1.64922791e-01]
 [ 3.18829561e-01  4.86189124e-01 -4.93693633e-01]
 [ 4.35050639e-02  7.92156024e-01  1.08891970e-02]
 [ 9.12882764e-04  3.09745271e-01  8.53784806e-01]]
Matrica E:
[[ 1.13717809  0.          0.        ]
 [ 0.         17.58269051  0.        ]
 [ 0.          0.          3.94408679]]
Matrica V.T:
[[ 0.83865173 -0.5087217  -0.19459061]
 [ 0.50026161  0.57814691  0.64458085]
 [-0.21541031 -0.63792506  0.73935784]]
Matrica A:
[[1. 3. 2.]
 [5. 6. 4.]
 [7. 8. 9.]
 [2. 1. 6.]]


In [10]:
# jos par primera

# Implementacija drugog algoritma

Naziva se Jakobijeva rotacija SVD. Poznata je i kao <b>dvostrana Jakobijeva metoda za SVD</b>.
Implementirali smo za simetrične matrice, ali modifikacija ovog algoritma radi i u slučajevima kada matrica nije simetrična. Algoritam koristi drugačiji kriterijum zaustavljanja od prethodnog i drugačije konstruiše 2x2 matricu na koju će biti primenjena Jakobijeva rotacija. 



In [11]:
def SVD(A, m, n, eps = 1.e-4):
    # 1. korak, B = A
    B = np.array(copy.copy(A))
    # 2. korak, U = I(mxn)
    U = np.eye(m,n) 
    # 3. korak, V + I(nxn)
    V = np.eye(n)
    
    # 4. korak, izračunava se N^2 = M, s = 0, first = true
    M = 0
    s = 0
    first = True
    for i in range(m):
        for j in range(n):
            M += B[i][j] * B[i][j]

    # korak 5
    a = 0
    b = 0
    c = 0
    error = 1
    while np.sqrt(s) > eps*eps*M or first == True:
        s = 0
        first = False
        
        for i in range(n-1):
            for j in range(i+1, n):
                s = s + B[i][j]*B[i][j] + B[j][i]*B[j][i]
                # izračunavanje c,s,d1,d2
                a = B[i][i]
                b = B[j][j]
                c = B[i][j]

                # Jakobijeva rotacija
                tau = ((b - a)* 1.0) / (2*c)
                t = np.sign(tau) * 1.0/(np.abs(tau) + np.sqrt(1 + tau*tau))
                cs = 1.0/np.sqrt(1 + t*t)
                sn = cs*t
                
                # apdejtujemo  2 po 2 podmatrice 
                B[i][i] = a - c*t
                B[j][j] = b + c*t
                B[i][j] = 0
                B[j][i] = 0
                
                # apdejtujemo ostatak i i j kolona
                for k in range(n):
                    if k != i and k != j:
                        tmp = B[i][k]
                        B[i][k] = cs*tmp - sn*B[j][k]
                        B[j][k] = sn*tmp + cs*B[j][k]
                        B[k][i] = B[i][k]
                        B[k][j] = B[j][k]
                # apdejtujemo sopstveni vektor matrice V
                for k in range(n):
                    tmp = V[k][i]
                    V[k][i] = cs*tmp - sn*V[k][j]
                    V[k][j] = sn*tmp + cs*V[k][j]
                    
                #  apdejtujemo sopstveni vektor matrice U
                for k in range(n):
                    tmp = U[k][i]
                    U[k][i] = cs*tmp - sn*U[k][j]
                    U[k][j] = sn*tmp + cs*U[k][j]
                
    # korak 5
    s = np.diag(B)
    S = np.eye(n)
    for i in range(n):
        S[i][i] = np.abs(s[i])
        if s[i] < 0:
            U[:,i] = -1*U[:,i]
    
    print("U je:")
    print(U)
    print("S je")
    print(S)
    print("V je:")
    print(V.T)
    
    print("A je:")
    print(np.dot(U, np.dot(S, V.T)))
    

# Primeri 

In [12]:
A = [ [1.0 ,5.0,7.0],
      [5.0 ,6.0,4.0],
      [7.0 ,4.0,9.0]]
U, S, V = la.svd(A)
print(U)
print(S)
print(V)
   
SVD(A,3,3)

[[-0.48441546  0.87476581 -0.01124407]
 [-0.50196102 -0.28844953 -0.81537231]
 [-0.71650317 -0.38933487  0.57882775]]
[ 16.53486179   3.76423709   3.2293753 ]
[[-0.48441546 -0.50196102 -0.71650317]
 [-0.87476581  0.28844953  0.38933487]
 [-0.01124407 -0.81537231  0.57882775]]
U je:
[[-0.87476581  0.01124407  0.48441546]
 [ 0.28844953  0.81537231  0.50196102]
 [ 0.38933487 -0.57882775  0.71650317]]
S je
[[  3.76423709   0.           0.        ]
 [  0.           3.2293753    0.        ]
 [  0.           0.          16.53486179]]
V je:
[[ 0.87476581 -0.28844953 -0.38933487]
 [ 0.01124407  0.81537231 -0.57882775]
 [ 0.48441546  0.50196102  0.71650317]]
A je:
[[ 1.  5.  7.]
 [ 5.  6.  4.]
 [ 7.  4.  9.]]


In [ ]:
#Jos primera 